In [1]:
# !pip install geemap

Defaulting to user installation because normal site-packages is not writeable


# Machine Learning with Earth Engine - Unsupervised Classification

### การจำแนกประเภทที่ไม่ต้องการการควบคุม (Unsupervised classification algorithms) ที่มีให้ใช้ใน Google Earth Engine

Source: https://developers.google.com/earth-engine/clustering

แพ็คเกจ ee.Clusterer จัดการกับการจำแนกประเภทที่ไม่ต้องการการควบคุม (หรือการจัดกลุ่ม) ใน Earth Engine ขั้นตอนวิธีเหล่านี้ใช้อัลกอริทึมที่ใช้ชื่อเดียวกันใน Weka ในปัจจุบัน รายละเอียดเพิ่มเติมเกี่ยวกับแต่ละ Clusterer มีให้ในเอกสารอ้างอิงในโปรแกรม Code Editor

Clusterers ถูกใช้ในลักษณะเดียวกับ classifiers ใน Earth Engine ขั้นตอนการทำงานทั่วไปสำหรับการจัดกลุ่ม (clustering) คือ:

รวบรวมคุณสมบัติที่เป็นตัวเลขของฟีเจอร์ที่ต้องการหากลุ่มสร้างอ็อบเจ็กต์ clusterer และตั้งค่าพารามิเตอร์ตามที่จำเป็น (ถ้ามี) training  clusterer โดยใช้ข้อมูลการฝึกประยุกต์ใช้ clusterer กับภาพหรือคอลเลกชันของ featureตั้งชื่อกลุ่มข้อมูลการฝึกคือ FeatureCollection ที่มีคุณสมบัติที่จะใช้เป็นข้อมูลนำเข้าให้กับ clusterer ต่างจาก classifiers ไม่มีค่าคลาสนำเข้าสำหรับ clusterer เหมือน classifiers ข้อมูลสำหรับขั้นตอนการฝึกและประยุกต์ใช้จะต้องมีจำนวนค่าเท่ากัน เมื่อ clusterer ที่ถูก training dataset แล้วถูกประยุกต์ใช้กับภาพหรือตาราง มันจะกำหนดรหัสกลุ่มเป็นจำนวนเต็มให้กับแต่ละพิกเซลหรือ feature

นี่คือตัวอย่างง่าย ๆ ในการสร้างและใช้ ee.Clusterer: 

![](https://i.imgur.com/IcBapEx.png)

## บทแนะนำขั้นตอนทีละขั้นตอน

### Import libraries

In [1]:
# นำเข้าแพ็กเกจ ee เพื่อให้สามารถใช้งาน Google Earth Engine API ได้
import ee

# นำเข้าแพ็กเกจ geemap เพื่อให้สามารถใช้งานเครื่องมือแสดงแผนที่จาก Google Earth Engine ได้
import geemap

### สร้างแผนที่โดยใช้ไลบรารี geemap และแสดงผลแผนที่

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### เพิ่มข้อมูลลงในแผนที่

In [3]:
# กำหนดจุดที่ตั้งข้อมูลแผนที่พื้นที่ศึกษาจาก FAO กรองด้วยการใช้พิกัดภูมิศาสตร์ 
point = ee.Geometry.Point([100.25181617367595, 16.819711283484004])  # อำเภอเมืองพิษณุโลก 

# โหลดข้อมูลสำหรับกรองพื้นที่อำเภอเมืองพิษณุโลกด้วยจุดพิกัด point
Mueang = ee.FeatureCollection('FAO/GAUL/2015/level2').filterBounds(point)
                    
# โหลดข้อมูลภาพถ่ายดาวเทียม Landsat-8 และกรองข้อมูลภาพ
image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2020-08-01', '2020-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

# ตัดขอบเขตพื้นที่ศึกษาของภาพถ่ายดาวเทียม
image_M = image.clip(Mueang)

# สร้างแผนที่โฟกัสโดยกำหนดพิกัดศูนย์กลางและระดับการซูม
Map.centerObject(Mueang, 8)

# เพิ่มชั้นข้อมูล การผสมสีข้อมูลจากดาวเทียม (Band Combinations) ลงในแผนที่
# ลักษณะของสิ่งปกคลุมดินจะปรากฏคล้ายกับการมองเห็นของมนุษย์ พืชหรือต้นไม้ที่มีสุขภาพดีจะมีสีเขียว ถนนเป็นสีเทา
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'],'min':0, 'max': 3000}, 'ภาพผสมสีธรรมชาติ (Red, Green, Blue)')

# การผสมรูปแบบนี้มีความใกล้เคียงกับการผสมสีธรรมชาติ แตกต่างที่เมืองจะสามารถแยกได้ชัดเจนมากกว่า
Map.addLayer(image, {'bands': ['B7', 'B6', 'B4'],'min':0, 'max': 3000}, 'ความเป็นเมือง (SWIR2, SWIR1, Red)')

# รูปแบบนี้ได้รับความนิยมอย่างมาก เพื่อศึกษาเกี่ยวกับพืชพรรณโดยเฉพาะ เนื่องจากพืชจะปรากฏเป็นสีแดงแยกกับวัตถุชนิดอื่นๆ อย่างชัดเจน
Map.addLayer(image, {'bands': ['B5', 'B4', 'B3'],'min':0, 'max': 3000}, 'สีอินฟราเรด (NIR, Red, Green)') 

# เป็นเทคนิคที่ใช้กันอย่างแพร่หลายในการวิเคราะห์ภาพดาวเทียมเพื่อทำการแยกแยะแหล่งน้ำและพื้นที่บนบก
Map.addLayer(image, {'bands': ['B5', 'B6', 'B4'],'min':0, 'max': 3000}, 'แยกแยะระหว่างพื้นที่บนบกและน้ำ(NIR, SWIR1, Red)') 

# การผสมในรูปแบบนี้มีประโยชน์ในการตรวจสอบพืชผลการเกษตร พืชที่แข็งแรงสมบูรณ์จะแสดงผลสีเขียวสดใส ส่วนที่รกล้างว่างเปล่าจะแสดงเป็นสีน้ำตาลหรือสีม่วง
Map.addLayer(image, {'bands': ['B6', 'B5', 'B2'],'min':0, 'max': 3000}, 'เกษตรกรรม (SWIR1, NIR, Blue)') 

# เพิ่มแผนที่ควบคุมเพื่อให้ผู้ใช้สามารถเปลี่ยนแปลงและเลือกชั้นข้อมูลได้
Map.addLayerControl()

### ตรวจสอบคุณสมบัติของภาพ

In [4]:
props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 0.02,
 'CLOUD_COVER_LAND': 0.02,
 'EARTH_SUN_DISTANCE': 0.984783,
 'ESPA_VERSION': '2_23_0_1b',
 'GEOMETRIC_RMSE_MODEL': 8.03,
 'GEOMETRIC_RMSE_MODEL_X': 5.773,
 'GEOMETRIC_RMSE_MODEL_Y': 5.582,
 'IMAGE_DATE': '2020-12-10',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_130048_20201210_20210313_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1615638742000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2020-12-10T03:43:30.3751370Z',
 'SOLAR_AZIMUTH_ANGLE': 150.658218,
 'SOLAR_ZENITH_ANGLE': 45.882446,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 130,
 'WRS_ROW': 48,
 'system:asset_size': '624.158094 MB',
 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_130048_20201210',
 'system:index': 'LC08_130048_20201210',
 'system:time_end': '2020-12-10 03:43:30',
 'system:time_start': '2020-12-10 03:43:30',
 'system:version': 1615882697

In [5]:
props.get('IMAGE_DATE').getInfo()

'2020-12-10'

In [6]:
props.get('CLOUD_COVER').getInfo()

0.02

### สร้างชุดข้อมูลสำหรับจุด training dataset

มีวิธีหลายวิธีที่คุณสามารถสร้างพื้นที่เพื่อสร้างชุดข้อมูลการสุ่มจุด training area ได้

- วาดรูปทรง (เช่น สี่เหลี่ยม) บนแผนที่และใช้ region = Map.user_roi
- กำหนดรูปทรงเรขาคณิต เช่น region = ee.Geometry.Rectangle([100.25181617367595,16.819711283484004,100.25472150345898,16.8008333638423])
- สร้างโซน buffer รอบจุด เช่น region = ee.Geometry.Point([100.25181617367595,16.819711283484004]).buffer(7000)

หากไม่กำหนดพื้นที่ มันจะใช้รอบขอบภาพเป็นค่าเริ่มต้นโดยอัตโนมัติ

In [7]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([100.25181617367595,16.819711283484004,100.25472150345898,16.8008333638423])
# region = ee.Geometry.Point([100.25181617367595,16.819711283484004]).buffer(7000)

In [8]:
# ข้อมูลสำหรับจุด training dataset บนภาพ
training = image.sample(
    **{
        'scale': 30,  # กำหนดขนาดพิกเซลที่ใช้ในการสุ่ม
        'numPixels': 5000,  # กำหนดจำนวนพิกเซลที่ต้องการสุ่ม
        'seed': 0,  # กำหนดค่าเริ่มต้นสำหรับการสุ่ม
        'geometries': True,  # ตั้งค่าเป็น True เพื่อรวมเรขาคณิตของพิกเซลในชุดข้อมูล
    }
)
# เพิ่มชั้นข้อมูลจุด training dataset ลงในแผนที่
Map.addLayer(training, {}, 'จุดสุ่มพื้นที่ training ')

# แสดงแผนที่ 
Map  

Map(center=[16.83375603874749, 100.29338635392926], controls=(WidgetControl(options=['position', 'transparent_…

### Train the clusterer

In [9]:
# สร้างข้อมูลตัวอย่าง clusterer และ training dataset
n_clusters = 5  
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

### จำแนกชั้นข้อมูลจากภาพถ่ายดาวเทียม

In [13]:
Map = geemap.Map(center=[16.812037436738137, 100.30989048991965], zoom=11)

# แบ่งกลุ่มข้อมูลนำเข้าโดยใช้จุดสุ่มที่ training dataset แล้ว
result = image.cluster(clusterer)

# แสดงผลการแบ่งกลุ่มด้วยสีสุ่ม
Map.addLayer(result.randomVisualizer().clip(Mueang), {}, 'clusters')

# แสดงแผนที่
Map  


Map(center=[16.812037436738137, 100.30989048991965], controls=(WidgetControl(options=['position', 'transparent…

### กำหนดค่าคีย์และสีสำหรับแต่ละกลุ่มของชั้นข้อมูลที่ได้จากการแบ่งกลุ่ม (cluster)

In [14]:
# กำหนดคีย์และสีของค่าที่จะนำเข้าสำหรับแต่ละกลุ่มของชั้นข้อมูล
legend_key = ['One', 'Two', 'Three', 'Four', 'ect']
legend_color = ['#FF0000', '#00CC00', '#FFD700', '#BDB76B', '#0000FF']

# ปรับรูปแบบของแผนที่
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

# เพิ่มชั้นข้อมูลที่ตัดขอบเขตพื้นที่ศึกษาลงในแผนที่
Map.addLayer(
    result.clip(Mueang), {'min': 1, 'max': 5, 'palette': legend_color}, 'Labelled clusters'
)
# เพิ่มแผนที่ควบคุมเพื่อให้ผู้ใช้สามารถเปลี่ยนแปลงและเลือกชั้นข้อมูลได้
Map.addLayerControl()

# แสดงแผนที่
Map


Map(bottom=237601.0, center=[16.812037436738137, 100.30989048991965], controls=(WidgetControl(options=['positi…

### ฟังก์ชันการแสดงภาพผลลัพธ์

In [152]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### ส่งออกผลการจำแนกประเภทข้อมูลแบบไม่กำกับดูแล (Unsupervised classification)

ส่งออกผลลัพธ์ไปยังคอมพิวเตอร์โดยตรง:

In [154]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

In [155]:
geemap.ee_export_image(result, filename=out_file, scale=10)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Acer\Downloads\cluster.tif


ส่งออกผลลัพธ์ไปยัง Google Drive:

In [157]:
geemap.ee_export_image_to_drive(
    result, description='clusters', folder='export', scale=10,maxPixels = 1e13        
)